# Word-Level Text Generation with LSTM

In addition to making predictions, RNNs may also be used as generative models (can learn the sequences and then generate entirely new seqences). One of RNN variant, LSTM neural network has been recognized as a very successful tool when working with sequences of letters or words.

Let's examine performance of basic LSTM model on generating text of fairy tales.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd 'drive/MyDrive/Colab Notebooks/nlg_tales_generation'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/nlg_tales_generation


In [2]:
import functions as f

from Text import *
from LSTM_class import *

from keras import layers, models, optimizers

## Text preprocessing

The loaded text file contains the content of tales scraped from websites. By creating the instance of Text object, the text is quickly preprocessed and tokenized; by creating the instance of Sequence object the text is prepared for use in LSTM model.

In [3]:
path_train, path_test = 'data/train.txt', 'data/test.txt'

input_train = f.read_txt(path_train)

In [4]:
max_len = 4
step = 3

text_train = Text(input_train)
text_train.tokens_info()

seq_train = Sequences(text_train, max_len, step)
seq_train.sequences_info()

total tokens: 890750, distinct tokens: 25165
number of sequences of length 4: 296916


The text is split into sequences of length 4 (max_len parameter) with step 3. We can see that the first sequence of 4 words starts with the first (0-index) word and the second sequence starts after 3 words, so from the 4th word (3-index).

In [5]:
print(text_train.tokens[:10])
print(text_train.tokens_ind[:10], '\n')

np.array(seq_train.sequences[:2])

['Once', 'upon', 'a', 'time', 'there', 'lived', 'a', 'sultan', 'who', 'loved']
[23084, 5388, 8660, 263, 17275, 23956, 8660, 14335, 10634, 11630] 



array([[23084,  5388,  8660,   263],
       [  263, 17275, 23956,  8660]])

TextDataGenerator is a Python generator that outputs batches of data (sequences and corresponding next words). Since the vocabulary size is over 25k, it's impossible to fit all data to the memory and that's why batch generator is extremely useful.

In [6]:
batch_size = 4096

params = {
  'sequence_length': max_len,
  'vocab_size': len(text_train),
  'batch_size': batch_size,
  'shuffle': True
}

train_generator = TextDataGenerator(seq_train.sequences, seq_train.next_words, **params)

## Training the LSTM model

We'll build a simple model with one LSTM layer, dropout and dense layer with softmax activation (to return word probabilities).

In [7]:
def lstm_model(sequence_length, vocab_size, layer_size, embedding=False):
  model = models.Sequential()
  if embedding:
    model.add(layers.Embedding(vocab_size, layer_size))
    model.add(layers.LSTM(layer_size))    
  else:
    model.add(layers.LSTM(layer_size, input_shape=(sequence_length, vocab_size)))
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(vocab_size, activation='softmax'))
  return model

In [8]:
model = lstm_model(max_len, len(text_train), 512)

optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [9]:
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=40,
          verbose=1)

Epoch 1/40
72/72 [==============================] - 135s 2s/step - loss: 7.6544
Epoch 2/40
72/72 [==============================] - 130s 2s/step - loss: 5.5843
Epoch 3/40
72/72 [==============================] - 131s 2s/step - loss: 4.9627
Epoch 4/40
72/72 [==============================] - 131s 2s/step - loss: 4.4947
Epoch 5/40
72/72 [==============================] - 130s 2s/step - loss: 4.0160
Epoch 6/40
72/72 [==============================] - 131s 2s/step - loss: 3.5458
Epoch 7/40
72/72 [==============================] - 131s 2s/step - loss: 3.1083
Epoch 8/40
72/72 [==============================] - 131s 2s/step - loss: 2.7399
Epoch 9/40
72/72 [==============================] - 131s 2s/step - loss: 2.4194
Epoch 10/40
72/72 [==============================] - 131s 2s/step - loss: 2.1634
Epoch 11/40
72/72 [==============================] - 131s 2s/step - loss: 1.9340
Epoch 12/40
72/72 [==============================] - 131s 2s/step - loss: 1.7456
Epoch 13/40
72/72 [==================

In [10]:
model.save('data/out/lstm_model')

INFO:tensorflow:Assets written to: data/out/lstm_model/assets


INFO:tensorflow:Assets written to: data/out/lstm_model/assets


## Text generation with LSTM model

Generating text with LSTM model requires building the prediction loop which starts with choosing a prefix and setting the number of words to generate. Then we need to predict the next word using our LSTM model and use this word as part of the prefix for the next model input. The loop is executed until the expected number of words is generated.

In [ ]:
#model = models.load_model('data/out/lstm_model')

In [11]:
token2ind, ind2token = text_train.token2ind, text_train.ind2token

input_prefix = 'Once upon a time'
text_prefix = Text(input_prefix, token2ind, ind2token)

In [12]:
pred = ModelPredict(model, text_prefix, token2ind, ind2token, max_len)

In [13]:
temperatures = [1, 0.7, 0.4, 0.1]

for temperature in temperatures:
  print('temperature:', temperature)
  print(pred.generate_sequence(100, temperature=0.7))
  print('\n')

temperature: 1
Once upon a time there were a man and noble princess!' being so good- natured, and that morning, when he awoke, he found it in his hand it to the palace. The young wild hut a lived, on the bridge of two days. Every day when they had to be done for now. She is Wait an ass, and said to the merchant, and thou wilt have your bed has looked to take her up. A large man was lost in favor. The king be plunged and in the water


temperature: 0.7
Once upon a time there were a lot of mice, but could run see if he had been so good, for you will see someone there was. All the great wild Huldre; but I am very much mistaken. The is Moor- there. done as he, and his children wept, and he sat down in the forest. Go, and give They all the of people that I had gone his bed with the little giant, and she would draw none out of the mountain. This was the first thing that I was


temperature: 0.4
Once upon a time there was a father who had to be allowed to stay overnight, night as the far- the

## Text generation with LSTM model with Embedding layer

The previous model was taking as an input the sequences of words represented as one-hot vectors. In the second approach, we'll feed indexes of words to the model and train the Embedding layers which will create word representations.

In [17]:
batch_size_emb = 4096

params_emb = params.copy()
params_emb['embedding'] = True

train_generator_emb = TextDataGenerator(seq_train.sequences, seq_train.next_words, **params_emb)

In [18]:
model_emb = lstm_model(max_len, len(text_train), 512, embedding=True)
model_emb.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [19]:
model_emb.fit(train_generator_emb,
              steps_per_epoch=len(train_generator_emb),
              epochs=40,
              verbose=1)

Epoch 1/40
72/72 [==============================] - 31s 413ms/step - loss: 7.4319
Epoch 2/40
72/72 [==============================] - 30s 411ms/step - loss: 5.1867
Epoch 3/40
72/72 [==============================] - 29s 406ms/step - loss: 4.7109
Epoch 4/40
72/72 [==============================] - 29s 399ms/step - loss: 4.3334
Epoch 5/40
72/72 [==============================] - 29s 404ms/step - loss: 3.9703
Epoch 6/40
72/72 [==============================] - 29s 398ms/step - loss: 3.6161
Epoch 7/40
72/72 [==============================] - 29s 397ms/step - loss: 3.2614
Epoch 8/40
72/72 [==============================] - 29s 403ms/step - loss: 2.9379
Epoch 9/40
72/72 [==============================] - 29s 402ms/step - loss: 2.6587
Epoch 10/40
72/72 [==============================] - 29s 403ms/step - loss: 2.4155
Epoch 11/40
72/72 [==============================] - 29s 395ms/step - loss: 2.2170
Epoch 12/40
72/72 [==============================] - 29s 404ms/step - loss: 2.0368
Epoch 13/40
7

In [20]:
model_emb.save('data/out/lstm_model_emb')

INFO:tensorflow:Assets written to: data/out/lstm_model_emb/assets


INFO:tensorflow:Assets written to: data/out/lstm_model_emb/assets


In [ ]:
#model_emb = models.load_model('data/out/lstm_model_emb')

In [21]:
pred_emb = ModelPredict(model_emb, text_prefix, token2ind, ind2token, max_len, embedding=True)

In [22]:
temperatures = [1, 0.7, 0.4, 0.1]

for temperature in temperatures:
  print('temperature:', temperature)
  print(pred_emb.generate_sequence(100, temperature=0.7))
  print('\n')

temperature: 1
Once upon a time there a majesty a young ball my grand- good sister, this is good smile. I know a; if I am a minute to see under the tree back to the table.' So he soon I have a thicket. But if you behold me that I have not heard this they are my great will, and will we have already already already killed here and drink as a rock to; and, as He went, to look for the second, and told them to marry you.' Marya Morevna


temperature: 0.7
Once upon a time there was a great many people, a splendid food, and a man who were out, he returned to her hut and went out to his mouth and said : Little Muck, that I wanted to go to the king that I cannot refuse. And he bowed to the ground, and were about with the horse, and sang that he was thought to a request, and from it was stuck to her, you will see, and you will come young man with you,' he asked. What do you


temperature: 0.4
Once upon a time there was a great rock of a green fig, and they entered his garden and were looking. Li